In [9]:
# Read data from "data/data_2017_2021.csv" and "data/data_2022_2023.csv"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read data
data_2017_2021 = pd.read_csv("data/data_2017_2021.csv")
data_2022_2023 = pd.read_csv("data/data_2022_2023.csv")

# For the "Apparatus" column, replace "VT_1", "VT_2" by "VT1", "VT2"
data_2022_2023["Apparatus"] = data_2022_2023["Apparatus"].replace({"VT_1": "VT1", "VT_2": "VT2", "hb":"HB"})
# Filter out NaN in the "Score" column
data_2022_2023 = data_2022_2023[data_2022_2023["Score"].notna()]

# Split data_2022_2023 by Gender
men_athletes_data = data_2022_2023[data_2022_2023["Gender"] == "m"]
women_athletes_data = data_2022_2023[data_2022_2023["Gender"] == "w"]

# Split men_athletes_data and women_athletes_data by "Country" into 2 lists
men_countries = men_athletes_data["Country"].unique()
men_country_athlete_dict = {country: men_athletes_data[men_athletes_data["Country"] == country] for country in men_countries}

women_countries = women_athletes_data["Country"].unique()
women_country_athlete_dict = {country: women_athletes_data[women_athletes_data["Country"] == country] for country in women_countries}

# For men_country_athlete_dict and women_country_athlete_dict, select the top 9 countries with most entries
# Select the top 9 countries with the most entries for both men and women
k = 12
men_top_12_countries = men_athletes_data["Country"].value_counts().head(k).index.tolist()
women_top_12_countries = women_athletes_data["Country"].value_counts().head(k).index.tolist()
# Filter the men_country_athlete_dict and women_country_athlete_dict by the top 9 countries
men_country_athlete_dict_12 = {country: data for country, data in men_country_athlete_dict.items() if country in men_top_12_countries}
women_country_athlete_dict_12 = {country: data for country, data in women_country_athlete_dict.items() if country in women_top_12_countries}


# Determine a 5-person team for each country based on the top 5 athletes who appear most frequently, and split the data for each country based on the selected athletes
for country in men_country_athlete_dict_12:
    top_5_athletes = men_country_athlete_dict_12[country].groupby("FirstName").size().sort_values(ascending=False).head(5)
    men_country_athlete_dict_12[country] = men_country_athlete_dict_12[country][men_country_athlete_dict_12[country]["FirstName"].isin(top_5_athletes.index)]
for country in women_country_athlete_dict_12:
    top_5_athletes = women_country_athlete_dict_12[country].groupby("FirstName").size().sort_values(ascending=False).head(5)
    women_country_athlete_dict_12[country] = women_country_athlete_dict_12[country][women_country_athlete_dict_12[country]["FirstName"].isin(top_5_athletes.index)]

# For each country, each athlete, group the data by "Apparatus" and calculate the average score of each apparatus, then replace the original data with the new data (average score)
for country in men_country_athlete_dict_12:
    for athlete in men_country_athlete_dict_12[country]["FirstName"].unique():
        athlete_data = men_country_athlete_dict_12[country][men_country_athlete_dict_12[country]["FirstName"] == athlete]
        apparatus_scores = athlete_data.groupby("Apparatus")["Score"].mean()
        for apparatus in apparatus_scores.index:
            men_country_athlete_dict_12[country].loc[(men_country_athlete_dict_12[country]["FirstName"] == athlete) & (men_country_athlete_dict_12[country]["Apparatus"] == apparatus), "Score"] = apparatus_scores[apparatus]
        men_country_athlete_dict_12[country] = men_country_athlete_dict_12[country].drop_duplicates(subset=["FirstName", "Apparatus"], keep="first")
for country in women_country_athlete_dict_12:
    for athlete in women_country_athlete_dict_12[country]["FirstName"].unique():
        athlete_data = women_country_athlete_dict_12[country][women_country_athlete_dict_12[country]["FirstName"] == athlete]
        apparatus_scores = athlete_data.groupby("Apparatus")["Score"].mean()
        for apparatus in apparatus_scores.index:
            women_country_athlete_dict_12[country].loc[(women_country_athlete_dict_12[country]["FirstName"] == athlete) & (women_country_athlete_dict_12[country]["Apparatus"] == apparatus), "Score"] = apparatus_scores[apparatus]
        women_country_athlete_dict_12[country] = women_country_athlete_dict_12[country].drop_duplicates(subset=["FirstName", "Apparatus"], keep="first")

# identify all apparatuses
apparatuses = data_2022_2023["Apparatus"].unique().tolist()
men_apparatuses = men_athletes_data["Apparatus"].unique().tolist()
women_apparatuses = women_athletes_data["Apparatus"].unique().tolist()
print("men's Apparatuses:", men_apparatuses)
print("women's Apparatuses:", women_apparatuses)

qual_men_apparatuses = ['HB', 'PH', 'FX', 'PB', 'SR', 'VT']
qual_women_apparatuses = ['BB', 'FX', 'UB', 'VT']

men's Apparatuses: ['HB', 'PH', 'FX', 'PB', 'SR', 'VT1', 'VT', 'VT2']
women's Apparatuses: ['BB', 'FX', 'UB', 'VT1', 'VT', 'VT2']


In [10]:
# The men's qualifying stage, for each country, first select an athlete to compete on all apparatus. Then, for each apparatus, select the top 2 athletes with the highest score and calculate the average score of the these 3 athletes. If the apparatus's total entry is less than 3, then the total score is the average of this apparatus's entries. If the apparatus's total entry is 0, then the total score is 10.
# The apparatus list is ['HB', 'PH', 'FX', 'PB', 'SR', 'VT']
# Also return the selected athletes that compete on all apparatuses

apparatus_list = qual_men_apparatuses
men_qualifying_scores = []
selected_athletes = []

for country in men_country_athlete_dict_12:
    country_scores = []
    country_athlete_data = men_country_athlete_dict_12[country]
    selected_athlete = country_athlete_data.groupby("FirstName")["Score"].sum().sort_values(ascending=False).head(1).index[0]
    selected_athletes.append(selected_athlete)
    for apparatus in apparatus_list:
        apparatus_data = country_athlete_data[country_athlete_data["Apparatus"] == apparatus]
        if len(apparatus_data) == 0:
            country_scores.append(10)
        else:
            top_2_athletes = apparatus_data.groupby("FirstName")["Score"].mean().sort_values(ascending=False).head(2)
            top_3_athletes = pd.concat([top_2_athletes, country_athlete_data[country_athlete_data["FirstName"] == selected_athlete][country_athlete_data["Apparatus"] == apparatus]["Score"]])
            if len(top_3_athletes) < 3:
                country_scores.append(top_3_athletes.mean())
            else:
                country_scores.append(top_3_athletes.mean())
    men_qualifying_scores.append(country_scores)

men_qualifying_df = pd.DataFrame(men_qualifying_scores, columns=apparatus_list, index=men_country_athlete_dict_12.keys())
selected_athletes_df = pd.DataFrame({"Country": men_country_athlete_dict_12.keys(), "FirstName": selected_athletes})

# compute the selected athletes' average score for each apparatus
selected_athletes_scores = []
for athlete in selected_athletes_df["FirstName"]:
    athlete_data = men_athletes_data[men_athletes_data["FirstName"] == athlete]
    athlete_scores = []
    for apparatus in apparatus_list:
        apparatus_data = athlete_data[athlete_data["Apparatus"] == apparatus]
        if len(apparatus_data) == 0:
            athlete_scores.append(10)
        else:
            athlete_scores.append(apparatus_data["Score"].mean())
    selected_athletes_scores.append(athlete_scores)

selected_athletes_scores_df = pd.DataFrame(selected_athletes_scores, columns=apparatus_list)
selected_athletes_scores_df.index = selected_athletes_df["FirstName"]
men_selected_athletes_df = selected_athletes_scores_df.copy()

/var/folders/31/gv1h6_nn0tq3df8fj0szk7sw0000gn/T/ipykernel_56116/606317691.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  top_3_athletes = pd.concat([top_2_athletes, country_athlete_data[country_athlete_data["FirstName"] == selected_athlete][country_athlete_data["Apparatus"] == apparatus]["Score"]])
/var/folders/31/gv1h6_nn0tq3df8fj0szk7sw0000gn/T/ipykernel_56116/606317691.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  top_3_athletes = pd.concat([top_2_athletes, country_athlete_data[country_athlete_data["FirstName"] == selected_athlete][country_athlete_data["Apparatus"] == apparatus]["Score"]])
/var/folders/31/gv1h6_nn0tq3df8fj0szk7sw0000gn/T/ipykernel_56116/606317691.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  top_3_athletes = pd.concat([top_2_athletes, country_athlete_data[country_athlete_data["FirstName"] == selected_athlete][country_athlete_data["Apparatus"] == appar

In [11]:
apparatus_list = qual_women_apparatuses
women_qualifying_scores = []
selected_athletes = []

# compute the selected athletes' average score for each apparatuses
for country in women_country_athlete_dict_12:
    country_scores = []
    country_athlete_data = women_country_athlete_dict_12[country]
    selected_athlete = country_athlete_data.groupby("FirstName")["Score"].sum().sort_values(ascending=False).head(1).index[0]
    selected_athletes.append(selected_athlete)
    for apparatus in apparatus_list:
        apparatus_data = country_athlete_data[country_athlete_data["Apparatus"] == apparatus]
        if len(apparatus_data) == 0:
            country_scores.append(10)
        else:
            top_2_athletes = apparatus_data.groupby("FirstName")["Score"].mean().sort_values(ascending=False).head(2)
            top_3_athletes = pd.concat([top_2_athletes, country_athlete_data[country_athlete_data["FirstName"] == selected_athlete][country_athlete_data["Apparatus"] == apparatus]["Score"]])
            if len(top_3_athletes) < 3:
                country_scores.append(top_3_athletes.mean())
            else:
                country_scores.append(top_3_athletes.mean())
    women_qualifying_scores.append(country_scores)

women_qualifying_df = pd.DataFrame(women_qualifying_scores, columns=apparatus_list, index=women_country_athlete_dict_12.keys())
selected_athletes_df = pd.DataFrame({"Country": women_country_athlete_dict_12.keys(), "FirstName": selected_athletes})

# compute the selected athletes' average score for each apparatuses
selected_athletes_scores = []
for athlete in selected_athletes_df["FirstName"]:
    athlete_data = women_athletes_data[women_athletes_data["FirstName"] == athlete]
    athlete_scores = []
    for apparatus in apparatus_list:
        apparatus_data = athlete_data[athlete_data["Apparatus"] == apparatus]
        if len(apparatus_data) == 0:
            athlete_scores.append(10)
        else:
            athlete_scores.append(apparatus_data["Score"].mean())
    selected_athletes_scores.append(athlete_scores)

selected_athletes_scores_df = pd.DataFrame(selected_athletes_scores, columns=apparatus_list)
selected_athletes_scores_df.index = selected_athletes_df["FirstName"]
women_selected_athletes_df = selected_athletes_scores_df.copy()

/var/folders/31/gv1h6_nn0tq3df8fj0szk7sw0000gn/T/ipykernel_56116/2280364166.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  top_3_athletes = pd.concat([top_2_athletes, country_athlete_data[country_athlete_data["FirstName"] == selected_athlete][country_athlete_data["Apparatus"] == apparatus]["Score"]])
/var/folders/31/gv1h6_nn0tq3df8fj0szk7sw0000gn/T/ipykernel_56116/2280364166.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  top_3_athletes = pd.concat([top_2_athletes, country_athlete_data[country_athlete_data["FirstName"] == selected_athlete][country_athlete_data["Apparatus"] == apparatus]["Score"]])
/var/folders/31/gv1h6_nn0tq3df8fj0szk7sw0000gn/T/ipykernel_56116/2280364166.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  top_3_athletes = pd.concat([top_2_athletes, country_athlete_data[country_athlete_data["FirstName"] == selected_athlete][country_athlete_data["Apparatus"] == ap

In [12]:
# select the top 8 countries with the highest total score on all apparatuses for men and women, separately
men_total_scores = men_qualifying_df.sum(axis=1)
women_total_scores = women_qualifying_df.sum(axis=1)

top_8_men_countries = men_total_scores.sort_values(ascending=False).head(8).index.tolist()
top_8_women_countries = women_total_scores.sort_values(ascending=False).head(8).index.tolist()


# Now for the team final, we just sort the top 8 countries by their total score on all apparatuses, for men and women separately, and arrange them in dataframes with columns "Country", "Score", "Rank"
# Create dataframes for the top 8 men's and women's countries with columns "Country", "Score", "Rank"
men_team_final_df = pd.DataFrame({"Score": men_total_scores[top_8_men_countries], "Rank": range(1, 9)})
women_team_final_df = pd.DataFrame({"Score": women_total_scores[top_8_women_countries], "Rank": range(1, 9)})

print("men's team final outcome:", men_team_final_df)
print("women's team final outcome:", women_team_final_df)

men's team final outcome:          Score  Rank
JPN  86.370000     1
GBR  82.439026     2
ITA  82.140547     3
ESP  81.685296     4
UKR  81.408667     5
TUR  81.130706     6
HUN  80.744389     7
GER  80.664250     8
women's team final outcome:          Score  Rank
ITA  55.151166     1
JPN  54.131667     2
BEL  52.939333     3
NED  52.354333     4
AUS  52.114167     5
GBR  51.714033     6
FRA  51.675162     7
FIN  51.199333     8


In [13]:
# Combine women_selected_athletes_df and men_selected_athletes_df into a single dataframe and give and rank based on AverageScore
# selected_athletes_df = pd.concat([women_selected_athletes_df, men_selected_athletes_df])
# selected_athletes_df["Rank"] = selected_athletes_df["AverageScore"].rank(method="dense", ascending=False).astype(int)
# print("individual final", selected_athletes_df)

# append a total score column
women_selected_athletes_df["TotalScore"] = women_selected_athletes_df[qual_women_apparatuses].sum(axis=1)
men_selected_athletes_df["TotalScore"] = men_selected_athletes_df[qual_men_apparatuses].sum(axis=1)

# determine the men's ranking for each apparatus and append a "Rank" column for each apparatus
apparatus_list = qual_men_apparatuses
for apparatus in apparatus_list:
    # append a "Rank" column for each apparatus
    men_selected_athletes_df[apparatus + "Rank"] = men_selected_athletes_df[apparatus].rank(method="dense", ascending=False).astype(int)
# append a "TotalRank" column
men_selected_athletes_df["TotalRank"] = men_selected_athletes_df["TotalScore"].rank(method="dense", ascending=False).astype(int)
print("men's individual final:", men_selected_athletes_df)

# determine the women's ranking for each apparatus and append a "Rank" column for each apparatus
apparatus_list = qual_women_apparatuses
for apparatus in apparatus_list:
    women_selected_athletes_df[apparatus + "Rank"] = women_selected_athletes_df[apparatus].rank(method="dense", ascending=False).astype(int)

# append a "TotalRank" column
women_selected_athletes_df["TotalRank"] = women_selected_athletes_df["TotalScore"].rank(method="dense", ascending=False).astype(int)

print("women's individual final:", women_selected_athletes_df)

men's individual final:                       HB         PH         FX         PB         SR       VT  \
FirstName                                                                       
Joel           13.174625  12.113000  13.694167  13.099600  12.916333  14.7000   
Nicola         12.721667  12.871833  14.012250  13.559800  13.077333  10.0000   
Mehmet Ayberk  12.077333  11.774667  12.991500  12.466333  14.115909  13.7000   
Jake           13.160077  13.152308  13.969800  13.646214  12.998200  15.0500   
James          12.963455  12.324077  13.205444  13.538778  13.238667  14.1665   
Krisztofer     12.924800  13.481400  14.124091  14.143000  12.959286  13.7000   
Andrin         12.049500  12.355333  13.555333  12.600000  12.783000  10.0000   
Leo            12.426200  12.579600  13.592800  13.986200  13.366400  10.0000   
Alexander      13.164304  12.634267  13.183143  12.685125  12.668500  14.2000   
Illia          13.676400  14.089667  13.559286  15.043706  13.075455  14.3000   
Wata